<a href="https://colab.research.google.com/github/leodenale/active-noise-cancelling/blob/master/anc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Active noise cancelling in Python

In [5]:
# Install pyaudio package
!apt-get install python-pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libportaudio2
Suggested packages:
  python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 python-pyaudio
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 88.6 kB of archives.
After this operation, 324 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-pyaudio amd64 0.2.11-1build2 [24.1 kB]
Fetched 88.6 kB in 1s (92.5 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 134443 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6

*Import* the necessary libraries

In [6]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError

ModuleNotFoundError: ignored

Add this line to make sure the matplotlib graph opens in a seperate window. Else real-time plotting would be impossible.

In [0]:
%matplotlib tk

Declaring the constants
* CHUNK: Samples per frame
* FORMAT: Audio format
* CHANNEL: Single channel for microphone (Multi-channel won't work in this program)
* RATE: Samples per second (44,1kHz is the most common)


In [0]:
CHUNK = 1024 * 2             
FORMAT = pyaudio.paInt16     
CHANNELS = 1                  
RATE = 44100

Use the PyAudio library to create an audio input and output stream with the previously declared constants.

In [0]:
p = pyaudio.PyAudio()

inputstream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

outputstream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

Prepare a matplotlib plot, where `line` resembles the input stream, and `line_mirror` the input stream mirrored in `y=128` (the anti sound used to cancel the input sound).

In [0]:
x = np.arange(0, 2 * CHUNK, 2)

line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=1, mec=(255, 0, 0, 1))
line_mirror, = ax.plot(x, np.random.rand(CHUNK), '-', lw=1, mec=(0, 0, 255, 1))

ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('amplitude')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks=[], yticks=[])

plt.show(block=False)

Declaring some values to mease the frame rate of the plot later.

In [0]:
print('stream started')

frame_count = 0
start_time = time.time()

In the first block the inputstream is converted from binary, to an integer list, to a numpy array. Then the anti sound is calculated and converted back to binary. And finally `outputstream.write` outputs the anti sound to the speakers.<br/>
In the second block the lines of the sound and anti sound as well as the plot itself are displayed. <br/>
In the third block the framerate is calculated and printed when the program ends.

In [0]:
while True:

    data = inputstream.read(CHUNK)
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    data_np = np.array(data_int, dtype='b')[::2] + 128
    data_np_mirror = 128 - (data_np - 128)
    data_int_mirror = (data_np_mirror - 128).tolist()
    data_mirror = struct.pack(str(CHUNK) + 'h', *data_int_mirror)
    outputstream.write(data_mirror)

    line.set_ydata(data_np)
    line_mirror.set_ydata(data_np_mirror)
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

    except TclError:
        frame_rate = frame_count / (time.time() - start_time)
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break